# Running API Demo on Colab

## Get necessary files
Clone the repo and download the model


In [2]:
!git clone https://github.com/nuwandda/create-ads-stable-diffusion.git
%cd create-ads-stable-diffusion
!wget https://huggingface.co/spaces/nuwandaa/adcreative-demo-api/resolve/main/weights/realisticVisionV60B1_v20Novae.safetensors\?download\=true --directory-prefix weights --content-disposition

Cloning into 'create-ads-stable-diffusion'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 70 (delta 30), reused 50 (delta 17), pack-reused 0
Receiving objects: 100% (70/70), 277.47 KiB | 21.34 MiB/s, done.
Resolving deltas: 100% (30/30), done.
[Errno 2] No such file or directory: 'adcreative-demo-api'
/content
--2024-01-06 17:44:19--  https://civitai.com/api/download/models/245598
Resolving civitai.com (civitai.com)... 104.18.22.206, 104.18.23.206, 2606:4700::6812:16ce, ...
Connecting to civitai.com (civitai.com)|104.18.22.206|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://civitai-delivery-worker-prod.5ac0637cfd0766c97916cefa3764fbdf.r2.cloudflarestorage.com/model/26957/realisticVisionV60NV.yY6Q.safetensors?X-Amz-Expires=86400&response-content-disposition=attachment%3B%20filename%3D%22realisticVisionV60B1_v60B1VAE.safetensors%22&X-

## Install dependencies
Install required packages

In [1]:
!pip install nest-asyncio pyngrok uvicorn
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.0 MB ? eta -:--:--ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 4.3/10.0 MB 158.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 8.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
  Using cached pyngrok-7.0.5-py3-none-any.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

## Setup Ngrok
Please visit [Ngrok](https://ngrok.com/) and sign up. Then, copy your authentication code and paste it below.


In [ ]:
!ngrok authtoken 'your_auth_token'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## Run the server
Just run the cell below and visit the public link that will be in the console output. Visit the link adding '/docs' at the end of it. For example: https://30f3-35-225-173-194.ngrok-free.app/docs

In [ ]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import fastapi as _fapi
from pyngrok import ngrok
import nest_asyncio
import uvicorn

import schemas as _schemas
import services as _services
import io

app = FastAPI()


@app.get("/")
def read_root():
    return {"message": "Welcome to AdCreative API"}


# Endpoint to test the backend
@app.get("/api")
async def root():
    return {"message": "Welcome to AdCreative API"}


@app.post("/api/generate-image/")
async def generate_image(imgPromptCreate: _schemas.ImageCreate = _fapi.Depends()):

    image = await _services.generate_image(imgPrompt=imgPromptCreate)

    memory_stream = io.BytesIO()
    image.save(memory_stream, format="PNG")
    memory_stream.seek(0)
    return StreamingResponse(memory_stream, media_type="image/png")


@app.post("/api/add_elements")
async def generate_image(addElements: _schemas.AddElements = _fapi.Depends()):

    image = await _services.add_elements(addElements=addElements)

    memory_stream = io.BytesIO()
    image.save(memory_stream, format="PNG")
    memory_stream.seek(0)
    return StreamingResponse(memory_stream, media_type="image/png")


@app.post("/api/create_ad")
async def generate_image(adCreate: _schemas.CreateAd = _fapi.Depends()):

    image = await _services.create_ad(adPrompt=adCreate)

    memory_stream = io.BytesIO()
    image.save(memory_stream, format="PNG")
    memory_stream.seek(0)
    return StreamingResponse(memory_stream, media_type="image/png")


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [755]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://27a2-35-225-68-151.ngrok-free.app
INFO:     212.253.199.44:0 - "GET / HTTP/1.1" 200 OK
INFO:     212.253.199.44:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     212.253.199.44:0 - "GET /openapi.json HTTP/1.1" 200 OK


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:     212.253.199.44:0 - "POST /api/create_ad?prompt=Cardboard%20Coffee%20Cup&hex_color=%23F8BFD4&punchline_text=Yeni%20d%C3%BCzene%20ho%C5%9F%20geldiniz.&button_text=Bas%C4%B1n%C4%B1z&punchline_color=%23F8BFD4&button_color=%23F8BFD4 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [755]
